# 콘텍스트 관리자와 else 블록

이 장에서는 다음 제어 기능을 살펴본다.

* with 문과 콘텍스트 관리자
* for, while, try 문에서의 else 블록

## 15.1 이것 다음에 저것: if 문 이외에서의 else 블록

else문은 if 문뿐만 아니라 for, while, try 문에서도 사용할 수 있다. 규칙은 다음과 같다.

* for : for 루프가 완전히 끝난 후(break는 안됨) else 블록 실행
* while : 조건식이 거짓이 되어 while이 끝난 후(break는 안됨) else 블록 실행
* try : try 블록에서 예외가 발생하지 않을 때만 실행되며, else에서 발생한 예외는 else 앞 except 블록에서 처리되지 않음

그러므로 예외, return, break, continue 문이 복합문을 빠져나오게 만들면 else 블록은 실행되지 않는다. 일반적인 루프에서의 else는 다음 패턴을 따른다.

```python
for item in my_list:
    if item.flavor == 'banana':
        break
else:
    raise ValueError('No banana flavor found!')
```

try/except 블록은 다음 처럼 구현한다.

```python
try:
    dangerous_call() # 예외가 발생할 수 있는 코드
except OSError:
    log('OSError...')
else:
    after_call() # 예외가 없을 때만 실행
```

이런 파이썬의 특성은 `EAFP`(Easier to Ask for Forgiveness than Permission)라는 슬로건으로 표현된다.

## 15.2 콘텍스트 관리자와 with 블록

콘텍스트 관리자 객체는 with 문을 제어하기 위해 존재하며, try/finally 패턴을 단순화한다. 콘텍스트 관리자 프로토콜은 `__enter__()`와 `__exit__()` 메서드로 구성된다. with 문이 시작될 때 `__enter__()` 메서드가 호출되며, 이 메서드는 with 블록의 끝에서 finally 절의 역할을 수행한다.

파일을 닫는 예제이다.

In [4]:
with open('vector_v7.py') as fp:
    src = fp.read(60)
    
len(src)

60

In [9]:
fp # fp 변수는 여전히 살아 있음

<_io.TextIOWrapper name='vector_v7.py' mode='r' encoding='UTF-8'>

In [6]:
fp.closed, fp.encoding

(True, 'UTF-8')

In [10]:
fp.read(60) # 닫혀있으므로 읽을 수 없다

ValueError: I/O operation on closed file.

In [8]:
src

'# vector_v7.py\n\nfrom array import array\nimport reprlib\nimpor'

콘텍스트 관리자 객체는 with 문 뒤의 표현식을 평가한 결과지만, as 절에 있는 타깃 변수의 값은 콘텍스트 관리자 객체의 `__enter__()` 호출 결과다.

제어 흐름이 with 문을 빠져나온 후에는 `__enter__()` 메서드가 반환한 객체가 아니라 `__exit__()` 메서드가 호출된다.

In [16]:
class LookingGlass:
    
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write # 나중에 사용하기 위해 객체 속성에 원래 sys.stdout.write() 메서드 객체를 저장
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY' # 타겟 변수에 무언가를 저장히기 위해 문자열 반환
    
    def reverse_write(self, text): 
        # text 인수를 거꾸로 뒤집고 나서 원래 sys.stdout.write() 메서드 호출
        self.original_write(text[::-1])
        
    def __exit__(self, exc_type, exc_value, traceback): 
        # 정상적으로 수행이 완료되면 파이썬은 None, None, None 인수로 __exit__() 메서드를 호출
        import sys
        sys.stdout.write = self.original_write # sys.stdout.write() 를 원래 메서드로 변경
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True # 예외가 처리되었음을 알려준다.
        # __exit__() 가 None이나 True 이외의 값을 반환하면 with 블록에서 발생한 예외가 상위 코드로 전달됨
    
    

In [19]:
from mirror import LookingGlass
with LookingGlass() as what: # __enter__() 메서드의 반환값을 what에 바인딩
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [20]:
what # 이제 with가 끝났으니, __enter__() 가 반환해서 what에 저장한 문자열을 제대로 출력

'JABBERWOCKY'

In [21]:
print('Back to normal') # print()가 정상적으로 작동

Back to normal


파이썬 인터프리터는 `__enter__()` 메서드를 호출할 때는 `self`만 인수로 전달한다. `__exit__()` 메서드를 호출할 때는 다음 세 인수를 전달한다.

* exc_type : ZeroDivisionError 등의 예외 클래스
* exc_value : 예외 객체
* traceback : traceback 객체

콘텍스트 관리자가 작동하는 방식을 자세히 알아보기 위해 with문 밖에서 LookingGlass 클래스를 사용하자.

In [22]:
from mirror import LookingGlass
manager = LookingGlass()
manager

In [27]:
monster = manager.__enter__()
monster == 'JABBERWOCKY' # monster에 저장된 문자열은 이게 맞으나

True

In [28]:
monster # sys.stdout 이 reversed 되어 있으므로 출력이 반대로 됨

'JABBERWOCKY'

In [25]:
manager

![LookingGLass](images/15_1.png)

## 15.3 contextlib 유틸리티

contextlib 모듈에는 다음과 같은 클래스와 함수가 있다.

* closing() : close() 메서드는 제공하지만 `__enter__()` / `__exit__()` 프로토콜을 구현하지 않는 객체로부터 콘텍스트 관리자를 생성하는 함수
* surpress : 지정한 예외를 임시로 무시하는 콘텍스트 관리자
* @contextmanager : 클래스를 생성하고 프로토콜을 구현하는 대신, 간단한 제너레이터 함수로부터 콘텍스트 관리자를 생성할 수 있게 해주는 데커레이터
* ContextDecorator : 콘텍스트 관리자를 함수 데커레이터로도 사용할 수 있게 해주는 기반 클래스
* ExitStack : ...

## 15.4 @contextmanager 사용하기